In [6]:
import pandas as pd

df = pd.read_csv("output/15_11_2021/features_output_15_11_2021_mini.csv")
print(len(list(df.columns)))

249


In [14]:
cols = list(df.columns)
liwc = list(filter(lambda x: "liwc" in x.lower(), cols))
mf = list(filter(lambda x: "foundation" in x.lower(), cols))

liwc_title = list(filter(lambda x: "liwc" in x.lower() and "title" in x.lower(), cols))
liwc_post = set(liwc).difference(set(liwc_title))

mf_title = list(filter(lambda x: "foundation" in x.lower() and "title" in x.lower(), cols))
mf_post = set(mf).difference(set(mf_title))

print(len(cols)) 
print(len(liwc)) # merged = 94
print("   {0}, {1}".format(len(liwc_title), len(liwc_post)))
print(len(mf))   # merged = 28
print("   {0}, {1}".format(len(mf_title), len(mf_post)))

cols_s = set(cols)
liwc_s = set(liwc)
mf_s = set(mf)
dif = cols_s.difference(liwc_s)
dif = dif.difference(mf_s)
print(dif)

249
188
   94, 94
56
   28, 28
{'post_id', 'writing_sty_?_count', 'post_num_comments', 'writing_sty_!_count', 'writing_sty_"_count'}
